# Experimento Doc2Vec - TFIDF

O objetivo aqui é usar doc2vec para classificação multi-classe no mini-dataset de atos/classes gerado após teste no ezTag. O resultado será comparado com uma representação textual TFIDF; ambos os casos serão testados com SVM e Logistic Regression.

Fontes principais:
- https://towardsdatascience.com/implementing-multi-class-text-classification-with-doc2vec-df7c3812824d
- https://github.com/UnB-KnEDLe/experiments/blob/master/members/matheus/BioC_XML_to_conll.ipynb
- https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html
- https://scikit-learn.org/stable/modules/generated/sklearn.svm.LinearSVC.html
- https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html

### Import de packages

In [1]:
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import utils
from tqdm import tqdm
import multiprocessing
import nltk

### Mini-Dataset de atos
- Instanciação em memória
- Retirada de classes com apenas 1 instância

In [2]:
import pandas as pd

df = pd.read_csv('/home/mstauffer/Documentos/UnB/9º Semestre/KnEDle/seg_textos/atos_segs.csv')
df

,tipo_ato,ato
0,nomeacao,NOMEAR LETÍCIA MATOS MAGALHÃES para exercer o ...
1,exoneracao,"EXONERAR, a pedido, ANNA LUIZA BARCIA HALLEY d..."
2,nomeacao,NOMEAR SANDRA TURCATO JORGE TOLENTINO para exe...
3,nomeacao,"NOMEAR RAIMUNDO DA COSTA SANTOS NETO, Procurad..."
4,ato_tornado_sem_efeito,TORNAR SEM EFEITO no Decreto de 02 de julho de...
...,...,...
268,substituicao,DESIGNAR\nrespectivamente TEREZA CRISTINA DE A...
269,substituicao,"Designar FLÁVIO DE ARAÚJO ALMEIDA, matrícula n..."
270,substituicao,"DESIGNAR OTAMÁ DANTAS BARRETO, matrícula\n159...."
271,retificacao,"RETIFICAR o Sexto Termo\nAditivo ao Contrato, ..."


In [3]:
df.tipo_ato.value_counts()

aposentadoria             69
nomeacao                  53
exoneracao                49
retificacao               38
substituicao              37
ato_tornado_sem_efeito    16
cessao                     9
abono_de_permanência       1
reversao                   1
Name: tipo_ato, dtype: int64

In [4]:
df = df[df.tipo_ato != 'abono_de_permanência']
df = df[df.tipo_ato != 'reversao']
df.tipo_ato.value_counts()

aposentadoria             69
nomeacao                  53
exoneracao                49
retificacao               38
substituicao              37
ato_tornado_sem_efeito    16
cessao                     9
Name: tipo_ato, dtype: int64

### Criação de listas de atos e tipos de ato, em ordem

In [5]:
atos = df.ato.to_list()
tipos = df.tipo_ato.to_list()

In [6]:
len(atos), len(tipos)

(271, 271)

### Retira \n das instâncias de atos

In [7]:
import re

at_contiguos = []
for ato in atos:
    at_contiguos.append(re.sub('\n', ' ', ato))
#at_contiguos

In [8]:
len(at_contiguos)

271

### Cria lista de listas de referência

infos é uma lista que contém duas outras listas:
- tipos
- at_contiguos

Ela servirá de referência para mapear instâncias

In [9]:
infos = []
infos.append(tipos)
infos.append(at_contiguos)
infos

infos[0][1]

'exoneracao'

In [10]:
i = 0
infos[0][i], infos[1][i]

('nomeacao',
 'NOMEAR LETÍCIA MATOS MAGALHÃES para exercer o Cargo em Comissão, Símbolo CC-04, código SIGRH B0001615, de Assessor Técnico, da Chefia de Gabinete, do Gabinete do Governador.')

### Função para tokenização

In [11]:
tqdm.pandas(desc="progress-bar")
# Function for tokenizing
def tokenize_text(text):
    tokens = []
    for sent in nltk.sent_tokenize(text):
        for word in nltk.word_tokenize(sent):
            if len(word) < 2:
                continue
            tokens.append(word.lower())
    return tokens

### Mapeia labels

In [12]:
possible_labels = df.tipo_ato.unique()
label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index

In [13]:
df['label'] = df.tipo_ato.replace(label_dict)
df

,tipo_ato,ato,label
0,nomeacao,NOMEAR LETÍCIA MATOS MAGALHÃES para exercer o ...,0
1,exoneracao,"EXONERAR, a pedido, ANNA LUIZA BARCIA HALLEY d...",1
2,nomeacao,NOMEAR SANDRA TURCATO JORGE TOLENTINO para exe...,0
3,nomeacao,"NOMEAR RAIMUNDO DA COSTA SANTOS NETO, Procurad...",0
4,ato_tornado_sem_efeito,TORNAR SEM EFEITO no Decreto de 02 de julho de...,2
...,...,...,...
268,substituicao,DESIGNAR\nrespectivamente TEREZA CRISTINA DE A...,4
269,substituicao,"Designar FLÁVIO DE ARAÚJO ALMEIDA, matrícula n...",4
270,substituicao,"DESIGNAR OTAMÁ DANTAS BARRETO, matrícula\n159....",4
271,retificacao,"RETIFICAR o Sexto Termo\nAditivo ao Contrato, ...",3


### Divisão treino e teste para fins de indexação do modelo doc2vec

In [14]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df.ato.values, 
                                                  df.label.values, 
                                                  test_size=0.25, 
                                                  random_state=14, 
                                                  stratify=df.label.values)

In [15]:
#X_train, X_test, y_train, y_test

In [16]:
infos[0][0], infos[1][0], len(infos[0])

('nomeacao',
 'NOMEAR LETÍCIA MATOS MAGALHÃES para exercer o Cargo em Comissão, Símbolo CC-04, código SIGRH B0001615, de Assessor Técnico, da Chefia de Gabinete, do Gabinete do Governador.',
 271)

In [17]:
i = 0
infos[0][i], infos[1][i]

('nomeacao',
 'NOMEAR LETÍCIA MATOS MAGALHÃES para exercer o Cargo em Comissão, Símbolo CC-04, código SIGRH B0001615, de Assessor Técnico, da Chefia de Gabinete, do Gabinete do Governador.')

In [18]:
# Initializing the variables
train_documents = []
test_documents = []

j = 0
for i in X_train:
    tag = y_train[j].item()
    train_documents.append(TaggedDocument(words=tokenize_text(i), tags=[tag]))
    j+=1

j = 0
for i in X_test:
    tag = y_test[j].item()
    test_documents.append(TaggedDocument(words=tokenize_text(i), tags=[tag]))
    j+=1

In [19]:
train_documents[7]

TaggedDocument(words=['exonerar', 'por', 'estar', 'sendo', 'nomeada', 'para', 'outro', 'cargo', 'karla', 'neres', 'de', 'laet', 'santana', 'do', 'cargo', 'em', 'comissão', 'símbolo', 'cc-06', 'código', 'sigrh', '07200232', 'de', 'assessor', 'do', 'gabinete', 'da', 'administração', 'regional', 'do', 'plano', 'piloto', 'do', 'distrito', 'federal'], tags=[1])

### Modelo doc2vec
- Instanciação
- Divisão treino e teste para uso no modelo em si
- Utilização do modelo com LogisticRegression

In [20]:
cores = multiprocessing.cpu_count()

model_dbow = Doc2Vec(dm=1, vector_size=100, negative=5, hs=0, min_count=2, sample = 0, workers=cores, alpha=0.025, min_alpha=0.001)
model_dbow.build_vocab([x for x in tqdm(train_documents)])
train_documents  = utils.shuffle(train_documents)
model_dbow.train(train_documents,total_examples=len(train_documents), epochs=30)

def vector_for_learning(model, input_docs):
    sents = input_docs
    targets, feature_vectors = zip(*[(doc.tags, model.infer_vector(doc.words, steps=20)) for doc in sents])
    return targets, feature_vectors

model_dbow.save('./actModel.d2v')

100%|██████████| 203/203 [00:00<00:00, 181351.16it/s]


In [21]:
len(train_documents)

203

In [22]:
y_train_model, X_train_model = vector_for_learning(model_dbow, train_documents)
y_test_model, X_test_model = vector_for_learning(model_dbow, test_documents)

As duas células abaixo são usadas para prevenir o seguinte: `DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().`
O método ravel() converte o shape do array para (n,). Ainda assim o warning persiste!

In [23]:
y_resampled = pd.DataFrame(y_test_model)
#y_resampled.values, type(y_resampled)

In [24]:
vai = y_resampled.values
vai = vai.ravel()
vai.shape

(68,)

In [25]:
#logreg = LogisticRegression(n_jobs=1, C=1e5, max_iter=300000)
logreg = LogisticRegression(max_iter=300000)
logreg.fit(X_train_model, y_train_model)
y_pred_d2v_reg = logreg.predict(X_test_model)
#print('Testing accuracy for acts %s' % accuracy_score(y_test_model, y_pred))
print('Testing accuracy for acts %s' % accuracy_score(vai, y_pred_d2v_reg))
#print('Testing F1 score for acts: {}'.format(f1_score(y_test_model, y_pred, average='weighted')))
print('Testing F1 score for acts: {}'.format(f1_score(vai, y_pred_d2v_reg, average='weighted')))

/home/mstauffer/.local/lib/python3.6/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


Testing accuracy for acts 0.8676470588235294
Testing F1 score for acts: 0.8590922855628739


In [26]:
from sklearn.svm import LinearSVC

svm_model_d2v = LinearSVC(max_iter=10000)
svm_model_d2v.fit(X_train_model, y_train_model)
svm_prediction_d2v = svm_model_d2v.predict(X_test_model)

/home/mstauffer/.local/lib/python3.6/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


### TFIDF + SVM

O primeiro passo é instanciar um objeto Tfidfvectorizer e gerar os vetores para o nosso corpus. A lista infos, que geramos algumas células atrás, é uma lista de 2 listas, que indexam em ordem label/texto de ato. Para gerar os vetores, vamos usar a lista dos textos de atos, infos[1].
- https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html

In [27]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(infos[1])
#print(vectorizer.get_feature_names())
print(X.shape)

(271, 1933)


Para os labels, vamos usar a coluna label do dataframe, gerada ao mapear os valores de infos[0], a lista de labels em ordem, para valores inteiros em um dict.

In [28]:
labels = df['label']
labels.shape

(271,)

Agora vamos dividir o corpus em treino e teste. Vamos usar como input X os vetores criados; usaremos o mesmo random state da divisão para o doc2vec.

In [29]:
X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.25, random_state=14, stratify=labels)

Agora temos a instanciação do modelo SVM em si.

In [30]:
svm_model = LinearSVC(max_iter=1000)
svm_model.fit(X_train, y_train)
svm_prediction_tfidf = svm_model.predict(X_test)

### TFIDF + regressão

In [31]:
#tfidf_logreg = LogisticRegression(n_jobs=1, C=1e5)
tfidf_logreg = LogisticRegression()
tfidf_logreg.fit(X_train, y_train)
y_pred_tfidf_reg = tfidf_logreg.predict(X_test)

### Síntese dos resultados

In [32]:
from sklearn.metrics import precision_recall_fscore_support

In [33]:
results = pd.DataFrame(columns = ['Precision', 'Recall', 'F1 score', 'support']
          )
results.loc['d2v + regressão'] = precision_recall_fscore_support(
          vai, 
          y_pred_d2v_reg, 
          average = 'weighted'
          )
results.loc['d2v + SVM'] = precision_recall_fscore_support(
          vai, 
          svm_prediction_d2v, 
          average = 'weighted'
          )
results.loc['TFIDF + SVM'] = precision_recall_fscore_support(
          y_test, 
          svm_prediction_tfidf, 
          average = 'weighted'
          )
results.loc['TFIDF + regressão'] = precision_recall_fscore_support(
          y_test, 
          y_pred_tfidf_reg, 
          average = 'weighted'
          )

In [34]:
results

,Precision,Recall,F1 score,support
d2v + regressão,0.857332,0.867647,0.859092,NaN
d2v + SVM,0.875908,0.867647,0.870539,NaN
TFIDF + SVM,0.986631,0.985294,0.984594,NaN
TFIDF + regressão,0.972976,0.970588,0.966309,NaN
